## Part 1 For Beginners Bag of Words

### What is NLP?

NLP(Natural Language Processing, 자연언어처리)는 텍스트 문제에 접근하기 위한 기법들의 집합입니다. 여기서는 IMDB movie review를 로딩하고 정제하는 것부터 시작해서, 단순한 Bag of Words 모델을 적용하여 어떤 리뷰가 "좋아요"인지 "싫어요"인지 상당히 정확하게 예측합니다.

### Befor you get started

이 튜토리얼은 파이썬을 기반으로 합니다. 만약 이전에 파이썬을 사용해 본 적이 없다면 Titanic competition python tutorial에 발을 담가 보기를 바랍니다. 만약 파이썬과 기초적인 NLP 기법에 이미 익숙하다면, part 2를 건너 뛰고 싶을 수도 있습니다. 

part 1은 플랫폼 의존적이지 않습니다. 이 튜토리얼 전반에 걸쳐 우리는 다양한 파이썬 모듈을 사용할 것이며, 이들은 텍스트 처리, 딥러닝, 랜덤 포레스트, 그리고 다른 application을 위한 것입니다. 더 자세한 정보를 알고 싶다면 "**Setting Up Your System**"를 확인하시기 바랍니다. 

### Reading the Data

Data 페이지에서 필수 파일들을 다운받을 수 있습니다. 첫 번째 파일은 `unlabeledTrainData`로, 25,000개의 IMDB 영화 리뷰를 포함합니다. 각 리뷰에는 긍정 또는 부정의 감성 라벨이 붙어 있습니다. 

다음으로, 탭으로 구분된 파일을 파이썬에 읽습니다. `pandas` 패키지가 제공하는 `read_csv` 함수를 이용하여 쉽게 데이터 파일을 읽고 쓸 수 있습니다. 

In [ ]:
import pandas as pd
train = pd.read_csv("labeledTrainData.tsv", header=0, delimiter = "\t", quoting = 3)

**header=0**은 파일의 첫 번째 행이 열 이름이라는 것을 가리킵니다. 
**delimiter=\t**는 자료가 탭으로 구분되었음을 나타냅니다. 
**quoting=3**은 python에게 큰따옴표를 무시하라고 하는 것입니다. 

훈련 데이터가 25,000개의 행과 3개의 열로 잘 구성되었는지 아래와 같이 확인할 수 있습니다. 

In [ ]:
print(train.shape)
train.columns.values

세 개의 열 이름은 각각 `"id", "sentiment", "array"`입니다. 
이제 몇 개의 리뷰를 살펴보기로 합니다. 

In [ ]:
print (train["review"][0])

리뷰에는 "",약어,구두점 등 HTML 태그가 포함되어 있습니다. 
- 이는 온라인으로부터 얻은 텍스트를 처리할 때 공통적으로 발생하는 문제입니다.

### Data Cleansing and Text Preprocessing

#### HTML 마크업을 제거하기: `BeautifulSoup` 패키지

In [ ]:
from bs4 import BeautifulSoup

#Initialize the BeautifulSoup object on a single movie review
example1 =  BeautifulSoup(train["review"][0])

#Print the raw review then the output of get_text(), for comparison

print(train["review"][0])

In [ ]:
print(example1.get_text())

`get_text()`는 태그와 마크업이 없는 리뷰의 텍스트를 제공합니다.

#### 구두점,숫자, stopword 다루기 : NLTK 과 정규표현

텍스트를 정제하는 방법에 대해 생각할 때, 우리는 우리가 해결해야 할 데이터 문제들에 대해 생각해 보아야 합니다. 많은 문제들에서, 구두점을 제거하는 것이 도움이 됩니다. 하지만 이 과제에서 우리는 감성 분석 문제를 다루고 있으며, "!!!"나 ":-(" 등이 감성을 전달할 수 있을 것이므로, 이들을 단어로 다루어야 합니다. 다만, 이 튜토리얼에서는 단순한 처리를 위해 구두점을 모두 제거합니다. 

이와 유사하게, 이 튜토리얼에서 우리는 숫자들을 제거하지만, 숫자들을 다른 방식으로 처리하여 더 많은 의미를 남겨둘 수도 있습니다. 예를 들어 우리는 숫자들을 단어로 다루거나 숫자들을 "NUM"으로 대체할 수 있습니다. 

구두점과 숫자들을 제거하기 위해, 우리는 정규표현을 다루는 패키지인 **`re`**를 사용합니다. 따로 설치할 필요는 없습니다. 

In [ ]:
import re

# Use regular expressions to do a find-and-replace

letters_only = re.sub("[^a-zA-Z]"," ", example1.get_text())

print (letters_only)

여기서 [^a-zA-Z]는 대소문자 알파벳이 아닌 모든 것의 집합을 가리킵니다. 

이번에는 리뷰의 대문자를 모두 소문자로 바꾸고, 개별 단어들로 쪼갭니다. 
(tokenization) 

In [ ]:
lower_case = letters_only.lower()  #소문자로 변환
words = lower_case.split() # 단어로 쪼개기

이번에는 그다지 많은 의미를 전달하지는 않지만 자주 나타나는 단어들을 다룰 차례입니다. 이러한 단어들은 "stopword"라고 하며, 영어에서 "a", "is", "the"와 같은 것입니다. 파이썬 `Natural Language Toolkit(NLTK)`로부터 stopword list를 불러올 수 있습니다. 

In [ ]:
import nltk
nltk.download()  # stopword를 포함한 텍스트 데이터 셋을 불러 옵니다. 
from nltk.corpus import stopwords # stopword list를 불러 옵니다. 
print(stopwords.words("english"))

In [ ]:
# "words"에서 stopword를 제거합니다. 
words = [w for w in words if not w in stopwords.words("english")]
print(words)

이외에도 데이터에 할 수 있는 것들이 많이 있습니다. 예를 들어, Poster Stemming과 Lemmatizing(NLTK에 있음)은 "messages","message", "messaging"을 같은 단어로 다룰 수 있게 합니다. 단순한 처리를 위해, 이 튜토리얼에서는 진행하지 않습니다. 

#### Putting it all together

지금까지 하나의 리뷰를 정제할 수 있는 코드를 작성하였습니다. 그러나, 우리는 25,000개의 훈련 리뷰를 정제하여야 하므로, 여러 번 실행할 수 있는 함수를 만들 필요가 있습니다.

In [ ]:
def review_to_words (raw_review):
    #원시 자료를 문자열로 변환하는 함수
    #입력은 하나의 문자열(리뷰 텍스트), 출력은 전처리된 하나의 문자열(리뷰 텍스트)
    # 1. HTML 제거
    review_text = BeautifulSoup(raw_review).get_text()
    # 2. 알파벳이 아닌 문자를 제거
    letters_only =  re.sub("[^a-zA-Z]"," ",review_text)
    # 3. 모두 소문자로 변환하고, 개별 단어로 쪼개기
    words = letters_only.lower().split()
    # 4. 파이썬에서, set을 검색하는 것은 list를 검색하는 것보다 훨씬 빠르기 때문에, stopwords를 집합으로 변환
    stops = set(stopwords.words("english"))
    # 5. stop words 제거
    meaningful_words = [w for w in words if not w in stops]
    # 6. 단어를 다시 하나의 문자열로 합치되, 스페이스로 구분하기
    # 결과를 반환하기 
    return(" ".join(meaningful_words))

이 함수에서 위에서 다루지 않았던 새로운 요소는 두 개가 있습니다. 
* stop word list 를 다른 데이터 타입인 set으로 변환하였습니다. 이는 속도 때문입니다. 
* 단어들을 하나의 문단으로 다시 결합하였습니다. 이는 결과물이 우리의 Bag of words에서 쉽게 사용될 수 있도록 하기 위한 것입니다. 

만약 하나의 리뷰에서 이 함수를 사용한다면, 

In [ ]:
clean_review = review_to_words (train["review"][0])
print(clean_review)

이제 반복문을 만들어서 모든 훈련셋을 한 번에 정제(cleansing)해 봅시다. 

In [ ]:
# 데이터프레임의 열 사이즈에 기초하여 리뷰 수를 구하기 
num_reviews = train["review"].size

# 리뷰 정제 결과를 담을 수 있도록 빈 리스트를 초기화하기
clean_train_reviews = []

# 각 리뷰에 대하여 반복하기; index i 가 0부터 훈련 셋의 리뷰 수까지 반복됨. 
for i in range(0, num_reviews) :
    # 실행되는 과정을 확인; 1000개마다(1000으로 나누었을 때 나머지가 0일 때마다) 프린트.
    if( (i+1)%1000 == 0 ):
        print("Review %d of %d\n" % ( i+1, num_reviews ))  
    # 각 리뷰에 대해 함수를 실행함. 결과를 clean_train_reviews에 추가함. 
    clean_train_reviews.append(review_to_words(train["review"][i]))

### Creating Feature from a Bag of Words (Using scikit-learn)

이번에는 정제된 텍스트 리뷰를 기계학습을 위한 숫자 표상으로 변환하는 방법을 알아봅니다. 여러 가지 방법이 있겠지만, 흔한 접근법 중 하나는 Bag of Words 입니다. Bag of Words 모델은 모든 문서로부터 어휘를 학습하며, 각 문서에서 각 단어들이 출현한 횟수를 세어 각 문서를 모델링합니다. 예를 들어, 두 개의 문장이 있다고 해 봅시다. 

* 문장 1: "The cat sat on the hat"
* 문장 2: "The dog ate the cat and the hat"

이 두 문장에서, 어휘는 다음과 같습니다. 

{ the, cat, sat, on, hat, dog, ate, and }

Bags of words를 얻기 위해, 우리는 각 문장에서 각 단어들이 나타난 횟수를 세어 줍니다. 
* 문장 1에서 "the"는 두 번, "cat", "sat", "on", "hat"은 각 한번 나타납니다. 따라서 문장 1의 벡터는 { 2, 1, 1, 1, 1, 0, 0, 0 }
* 문장 2의 벡터는 { 3, 1, 0, 0, 1, 1, 1, 1}
  
IMDB 데이터는 많은 수의 리뷰를 포함하므로, 이 데이터는 큰 어휘집을 제공할 것입니다. feature vector의 크기를 제한하기 위해, 우리는 최대 어휘집 크기를 선택해야 합니다. 아래에서, 우리는 5,000개의 가장 자주 나타나는 단어들을 사용합니다(stopword에 의해 제외된 것들은 빼고).  

이제 `scikit-learn`의 `feature_extraction` 모듈을 사용하여 `bag-of-words` feature를 만들 것입니다. 

In [ ]:
print ("Creating the bag of words...\n")
from sklearn.feature_extraction.text import CountVectorizer

#"CountVectorizer" 오브젝트를 초기화한다. scikit_learn의 bag of words 도구이다. 
vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             stop_words = None,
                             max_features = 5000)
# fit_transform()은 두 가지 기능이 있다 : 우선 이 함수는 모델을 적합하고, 어휘를 학습한다
# 다음으로, 이 함수는 우리의 훈련 데이터를 feature vector로 만든다. 
# fit_transform()의 입력은 반드시 문자열의 리스트여야 한다. 
train_data_features =  vectorizer.fit_transform(clean_train_reviews)

# 결과를 Numpy array로 변환한다.
train_data_features =  train_data_features.toarray()

훈련 데이터의 array가 이제 어떤 모양인지 보려면, 

In [ ]:
print(train_data_features.shape)

훈련 데이터의 array는 25,000개의 행과 5,000개의 feature(어휘집의 단어)로 구성됩니다.
덧붙여, `ContVectorizer` 안에도 전처리, tokenization, stopword 제거 옵션이 있음을 알아둡시다.  

이제 Bag of words 모델이 훈련되었으니 어휘집을 살펴봅니다.  

In [ ]:
vocab = vectorizer.get_feature_names()
print(vocab)

만약 관심이 있다면, 어휘집에서 각 단어의 빈도를 계산할 수도 있습니다. 

In [ ]:
import numpy as np

#어휘집의 각 단어의 갯수를 요약하기

dist = np.sum(train_data_features, axis=0)

# 각각에 대하여 어휘집의 단어와 훈련 셋에서의 빈도를 출력
for tag, count in zip(vocab, dist):
    print(count, tag)

### Randon Forest

 Bag of words로부터 숫자로 된 훈련 특성들이 만들어졌으며, 각 특성 벡터에는 감성 라벨이 붙어 있습니다. 이제 지도 학습을 진행합니다. 여기서 우리는 Random Forest Classifier를 사용합니다. 랜덤 포레스트 알고리즘은 scikit-learn에 포함되어 있습니다. 아래에서, 우리는 트리의 숫자의 default 값을 100으로 합니다. 트리의 수가 많아지면 성능이 좋아지기도 하지만 확실히 시간은 더 오래 걸립니다. 마찬가지로, 각 리뷰에 포함된 특성이 많을수록 시간이 많이 걸립니다.   

In [ ]:
print ("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier

# 100개의 Random Forest Classifier로 초기화
forest =  RandomForestClassifier(n_estimators = 100)

# 훈련 셋에 랜덤포레스트 모델 적합. Bag of words를 feature 로 사용하고, 감성 라벨을 결과 변수로 사용함. 
forest =  forest.fit(train_data_features, train["sentiment"])

### Creating a Submissions

이제 검증 셋에 훈련된 랜덤 포레스트를 실행하고, 제출 파일을 만듭니다. 

검증 셋에 대하여 Bag of words를 사용할 떄, 우리는 `fit_transform`이 아니라 `transform`을 사용합니다. 

기계 학습에서, 검증 셋은 모형을 적합하기 위해 사용해서는 안 됩니다. 과적합의 위험이 있기 때문입니다. 

In [ ]:
# 테스트 데이터 불러오기
test =  pd.read_csv("testData.tsv", header=0, delimiter="\t", quoting = 3)

# 25,000개의 행과 2개의 열이 있는지 확인
print(test.shape)

# 리뷰를 정제한 후 하나씩 추가하기 위한 빈 리스트 만들기
num_reviews = len(test["review"])
clean_test_reviews = []

print ("Cleansing and parsing the test set movie reviews...\n")

for i in range(0,num_reviews):
    if ((i+1) % 1000 == 0):
        print ("Review %d of %d\n" % (i+1, num_reviews))
    clean_review = review_to_words (test["review"][i])
    clean_test_reviews.append (clean_review)

# test set에 대한 bag of words 얻기, numpy array로 변환
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# 예측을 위해 랜덤포레스트 모델을 사용
result = forest.predict(test_data_features)

# id, sentiment 열과 함께 pandas 데이터프레임에 결과 복사
output = pd.DataFrame (data ={"id":test["id"], "sentiment":result})

# pandas를 쉼표로 구분된 csv에 쓰기
output.to_csv("Bag_of_Words_model.csv", index = False, quoting = 3)

축하합니다! 이제 첫 번째 제출을 위한 준비가 되었습니다. 결과를 바꾸기 위해 다른 것들도 시도해 보시기 바랍니다. 리뷰를 다른 방식으로 정제할 수도 있고, Bag of words 표상을 위한 어휘집의 숫자를 다르게 할 수도 있습니다. stemming을 쓸 수도 있고, 다른 분류기를 사용할 수도 있습니다. 